In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py39" --user
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
!unzip -q "/content/gdrive/MyDrive/Inputs.zip"

In [8]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
# %pip install torchsampler
import matplotlib.pyplot as plt
from pandas.core.common import flatten  
import numpy as np
import random
import torch
import os
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, Subset
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [9]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [10]:
p = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0 ,0 , 1, 1, 1, 1, 1, 1, 0, 0, 0])
p = torch.from_numpy(p)
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir,  transform=None):
        self.img_labels = annotations_file
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        image_matrix = np.load(os.path.join('Inputs/Pot_{}_Input_Seq.npy'.format(idx+1 if idx>=9 else '0'+str(idx+1))), allow_pickle=True)
        # image_matrix=np.vstack(image_matrix).astype(np.float64)
        image_matrix=np.array(image_matrix, dtype=[('O', np.float64)]).astype(np.float64)
        label = self.img_labels[idx]

        cropped_img_matrix = []
        if self.transform is not None:
          for image in image_matrix:
            # image = self.transform(image=image)["image"]
            # print(self.transform(image=image)["image"].shape)
            cropped_img_matrix.append(self.transform(image=image)["image"])
        my_tensor = torch.stack(cropped_img_matrix)
        # image_matrix = torch.from_numpy(image_matrix)

        
        return my_tensor, label

val_transform = A.Compose(
    [
        A.CenterCrop(height=224, width=224),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.9),
        # A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)



transformed_dataset = CustomImageDataset(annotations_file=p, img_dir='Inputs', transform=val_transform)

def weight_rand(images):
  n_images = len(images)
  count_per_class = [0] * 2
  for _,image_class in images:
      count_per_class[image_class] += 1
  weight_per_class = [0.] * 2
  for i in range(2):
      weight_per_class[i] = float(n_images) / float(count_per_class[i])
  weights = [0] * n_images
  for idx, (imgggg, image_class) in enumerate(images):
      weights[idx] = weight_per_class[image_class]
  # print(weight_per_class, count_per_class)

  return weights, weight_per_class, count_per_class




In [11]:
train_dataset = transformed_dataset
print(len(train_dataset))
my_subset = Subset(train_dataset, [0, 1, 2, 3, 4, 5, 12, 13 ,14, 18, 19, 20] )
test_dataset = Subset(train_dataset,[6, 7, 8, 9, 10, 11, 15, 16, 17, 21, 22, 23, 24, 25, 26, 27, 28, 29]) 



30


In [12]:
print(len(test_dataset))

18


In [13]:
# a = train_dataset[0]

In [14]:
class CNN_LSTM(nn.Module):
    def __init__(self, device):
        super(CNN_LSTM, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 3)
        self.pool1 = nn.MaxPool2d(2, 2)            
        self.conv2 = nn.Conv2d(20, 40, 3)       
        self.conv3 = nn.Conv2d(40, 60, 3)      
        self.conv4 = nn.Conv2d(60, 80, 3)   
        # self.conv5 = nn.Conv2d(80, 100, 3)   
        # self.conv6 = nn.Conv2d(100, 120, 3)   
        # self.conv7 = nn.Conv2d(120, 140, 3)   
 
        
        self.fc1 = nn.Linear(3000, 1000)       
        self.fc3 = nn.Linear(1000, 500)               
        self.fc4 = nn.Linear(500, 250)             
        self.fc5 = nn.Linear(250, 100)               
        self.fc6 = nn.Linear(100, 1)               
                                              
       

        self.lstm1 = nn.LSTM(80*12*12,3000)
        self.sigmoid = nn.Sigmoid()
        self.device = device


    def forward(self, x, hidden):
        for img in x:
            # y = torch.permute(img,(2,0,1)).float()
            y = img
            y = self.pool1(F.relu(self.conv1(y))) 
            y = self.pool1(F.relu(self.conv2(y))) 
            y = self.pool1(F.relu(self.conv3(y)))
            y = self.pool1(F.relu(self.conv4(y))) 
            # y = self.pool1(F.relu(self.conv5(y))) 
            # y = self.pool1(F.relu(self.conv6(y))) 
            # y = self.pool1(F.relu(self.conv7(y))) 
            out, hidden = self.lstm1(y.view(-1, 80*12*12), hidden)                     
        out = F.relu(self.fc1(out))
        # out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))
        out = F.relu(self.fc5(out))
        out = self.sigmoid(self.fc6(out))
        out = out.view(1, -1)
        out = out[:,-1]
        return out
        # x = torch.permute(x,(2,0,1)).float()
        # x = self.pool1(F.relu(self.conv1(x)))  
        # x = self.pool1(F.relu(self.conv2(x)))  
        # x = self.pool1(F.relu(self.conv3(x)))
        # x = self.pool1(F.relu(self.conv4(x)))
        # x = self.lstm_layer(x.view(-1, 120*65*118))
        # x = x.view(-1, 30*89*48)            # -> n, 400
        # return x
    def init_hidden(self):
        # c0 = torch.rand(100).float().to(self.device)
        # h0 = torch.rand(100).float().to(self.device)
        c0 = torch.zeros((1, 3000))
        h0 = torch.zeros((1, 3000))
        return (h0,c0)

In [15]:
# img = torch.permute(a[0][0],(2,0,1)).float()
model = CNN_LSTM(device)




In [16]:
# b = model(a[0], model.init_hidden())

In [17]:
lr=0.0001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
no_of_epochs = 10
counter = 0
loss_list = []
# model.train()
for epoch in range(no_of_epochs):
    h = model.init_hidden()
    running_loss = 0.0
    # for inputs, labels in train_dataset:
    for i, data in enumerate(my_subset, 0):
        print('------------------------------------------------------')
        inputs, labels = data
        inputs, labels = inputs.float(), labels.float()
        counter += 1
        h = tuple([e.data for e in h])
        model.zero_grad()
        output = model(inputs, h)
        print('Pot',counter)
        # print('Pot: ', output)

        print('Predicted Sigmoid',output.squeeze().item(),'Truth Label', labels.item())
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        print('Loss', loss.item())
        loss_list.append(loss.item())
        # if epoch == 50 or epoch == 60 or epoch == 70 or epoch == 80 or epoch == 90 :
        #   torch.save(model.state_dict(), './state_dict_{}.pt'.format(epoch))
        #   np.savez("./losslist_{}".format(epoch), np.array(loss_list))
        print('------------------------------------------------------')

print('done')

------------------------------------------------------
Pot 1
Predicted Sigmoid 0.5132983326911926 Truth Label 1.0
Loss 0.6668980717658997
------------------------------------------------------
------------------------------------------------------
Pot 2
Predicted Sigmoid 0.5128850340843201 Truth Label 1.0
Loss 0.6677035689353943
------------------------------------------------------
------------------------------------------------------
Pot 3
Predicted Sigmoid 0.5127367377281189 Truth Label 1.0
Loss 0.6679927706718445
------------------------------------------------------
------------------------------------------------------
Pot 4
Predicted Sigmoid 0.512779176235199 Truth Label 1.0
Loss 0.6679099798202515
------------------------------------------------------
------------------------------------------------------
Pot 5
Predicted Sigmoid 0.51297926902771 Truth Label 1.0
Loss 0.6675198674201965
------------------------------------------------------
--------------------------------------

In [ ]:
# Loading the best model
# model.load_state_dict(torch.load('./state_dict.pt'))
num_correct = 0
h = model.init_hidden()

# model.eval()
with torch.no_grad():
  for inputs, labels in test_dataset:
      h = tuple([each.data for each in h])
      inputs, labels = inputs, labels
      
      output = model(inputs, h)
      # test_loss = criterion(output.squeeze(), labels.float())
      # print('Test Loss : ', test_loss)
      print('Predicted Sigmoid: ', output)

      pred = torch.round(output.squeeze())  # Rounds the output to 0/1
      # correct_tensor = pred.eq(labels.float().view_as(pred))
      print('Predicted Label: ', pred)
      print('Truth: ', labels)
      # model.zero_grad()
      # correct = np.squeeze(correct_tensor.numpy())
      # num_correct += np.sum(correct)
      if int(pred.item()) == labels:
        num_correct += 1 

print("accuracy: ", num_correct/18) 